In [2]:
import os
import cv2
from src.preprocessing import make_square
from tqdm import tqdm
import random
import json

## No need to run again

In [27]:
filenames = os.listdir("data/cfp")

random.seed(42)
random.shuffle(filenames)
filenames[:10]

['DEV13781.jpg',
 'DEV12858.jpg',
 'DEV02899.jpg',
 'DEV12129.jpg',
 'DEV04490.jpg',
 'DEV10760.jpg',
 'DEV02890.jpg',
 'DEV05174.jpg',
 'DEV03301.jpg',
 'DEV03092.jpg']

In [13]:
original_shuffled_map = {}

for i in range(len(filenames)):
    original_shuffled_map[filenames[i]] = f"SHUF{str(i).zfill(5)}"

with open("original_shuffled_map.json", "w") as fp:
    json.dump(original_shuffled_map,fp) 

original_shuffled_map['DEV13781.jpg']

NameError: name 'filenames' is not defined

## Run this

In [3]:
with open('original_shuffled_map.json') as fp:
    original_shuffled_map = json.load(fp)

In [4]:
from multiprocessing import Pool

def _make_shuffled_square_img(filename):
    try:
        file = f"data/cfp/{filename}"
        img = cv2.imread(file)
        square_img = make_square(img)
        new_name = original_shuffled_map[filename]
        cv2.imwrite(f"data/shuffled_square/{new_name}.jpg", square_img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    except Exception as e:
        print(filename, e)
        
        
l_files = os.listdir("data/cfp")
with Pool() as pool:
    r = list(tqdm(pool.imap(_make_shuffled_square_img, l_files)))
    
print('Finished.')


2437it [01:50, 17.21it/s]

via_project_11Sep2022_17h57m_csv.csv axis 0 is out of bounds for array of dimension 0


2538it [01:53, 35.30it/s]

via_project_11Sep2022_17h57m.json axis 0 is out of bounds for array of dimension 0


15002it [10:39, 23.46it/s]

Finished.
